In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os


from pathlib import Path

In [2]:
# create a dictionary with all three CSV's
covid_data = { 
         "states" : { "file" : "Resources/original_states.csv"},
         "prison" : { "file" : "Resources/original_prison.csv"},
         "deaths" : {"file" :"Resources/original_deaths.csv"}
       }



In [3]:
#create a DataFrame for all three csv's

for k in covid_data:
    d = pd.read_csv(covid_data[k]["file"])
    covid_data[k]["df"] = d
    
states_df = covid_data["states"]["df"]
prison_df = covid_data["prison"]["df"]
deaths_df = covid_data["deaths"]["df"]



In [4]:
# Keep the columns that needed for the analysis
states_df = states_df.loc[:, ("date","state","cases","deaths")]
prison_df = prison_df.loc[:, ("name","total_prisoner_cases","total_prisoner_deaths","as_of_date")]
deaths_df = deaths_df.loc[:, ("state","date","cumulative_deaths")]

In [5]:
#rename columns 
states_df.rename(columns = {"date" : "Date","state":"State","cases":"Number_of_Cases", "deaths":"Number_of_Deaths"}, inplace = True)
prison_df.rename(columns = {"as_of_date":"Date", "name" : "State", "total_prisoner_cases":"Total_Prisoner_Cases","total_prisoner_deaths":"Total_Prisoner_Deaths", }, inplace = True)
deaths_df.rename(columns = {"date":"Date","state" : "State","cumulative_deaths":"Cumulative_Prisoner_Deaths"}, inplace = True)


In [6]:
# Reorganizing the columns
states_df = states_df[["Date","State", "Number_of_Cases", "Number_of_Deaths"]]
prison_df = prison_df[["Date", "State", "Total_Prisoner_Cases","Total_Prisoner_Deaths"]]
deaths_df = deaths_df[["Date","State","Cumulative_Prisoner_Deaths"]]

In [7]:
# drop the rows with missing data
clean_states_df = states_df.dropna(how = 'any')
clean_prison_df = prison_df.dropna(axis = 0, how = 'any')
clean_deaths_df = deaths_df.dropna(axis = 0, how = 'any')

In [8]:
# change the date format

clean_prison_df['Date'] = pd.to_datetime(clean_prison_df['Date'])
clean_states_df['Date'] = pd.to_datetime(clean_states_df['Date'])
clean_deaths_df[('Date')] = pd.to_datetime(clean_deaths_df[('Date')])
clean_states_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21134 entries, 0 to 21133
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              21134 non-null  datetime64[ns]
 1   State             21134 non-null  object        
 2   Number_of_Cases   21134 non-null  int64         
 3   Number_of_Deaths  21134 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 825.5+ KB


/Users/sarapatel/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/sarapatel/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [9]:
# create an index
clean_deaths_df = clean_deaths_df.set_index(['State', 'Date'])
clean_states_df = clean_states_df.set_index(['State', 'Date'])
clean_prison_df = clean_prison_df.set_index(['State', 'Date'])


In [10]:
# Check to see if there are any rows with missing data
# clean_death_df.count()
# clean_states_df.count()
# clean_prison_df.count()

In [11]:
# sort columns
clean_deaths_df = clean_deaths_df.sort_values(['State', 'Date'], ascending=False)
clean_states_df = clean_states_df.sort_values(['State', 'Date'], ascending=False)
clean_prison_df = clean_prison_df.sort_values(['State', 'Date'], ascending=False)

print(clean_states_df, clean_deaths_df, clean_prison_df)

                    Number_of_Cases  Number_of_Deaths
State   Date                                         
Wyoming 2021-03-21            55581               693
        2021-03-20            55581               693
        2021-03-19            55581               693
        2021-03-18            55479               693
        2021-03-17            55449               693
...                             ...               ...
Alabama 2020-03-17               39                 0
        2020-03-16               29                 0
        2020-03-15               23                 0
        2020-03-14               12                 0
        2020-03-13                6                 0

[21134 rows x 2 columns]                     Cumulative_Prisoner_Deaths
State   Date                                  
Wyoming 2021-03-07                       682.0
        2021-03-06                       682.0
        2021-03-05                       682.0
        2021-03-04                   

In [12]:
# hej not sure if we should merge them here or in the postgress?
# same for teh calculations?

merged_df = pd.merge(clean_states_df, clean_deaths_df, how='left', left_on=["State", "Date"], right_on=["State", "Date"])
merged_df = pd.merge(merged_df, clean_prison_df, how='left', left_on=["State", "Date"], right_on=["State", "Date"])



In [13]:
#save clean File to a csv
clean_deaths_df.to_csv("Resources/cleaned_deaths.csv")

In [14]:
#save clean File to a csv
clean_states_df.to_csv("Resources/cleaned_states.csv")

In [15]:
#save clean File to a csv
clean_prison_df.to_csv("Resources/cleaned_prison.csv")

In [16]:
#save merged to csv
merged_df.to_csv("Resources/cleaned_merged.csv")

In [17]:
from sqlalchemy import create_engine
# Connect to Postgres "prison" db
engine = create_engine('postgresql://localhost:5432/prison')



In [18]:
# Save DataFrames to Postgres DB 
# Added all data but only using merged table for analytics
try: 
    clean_prison_df.to_sql('prison', engine)
    clean_deaths_df.to_sql('deaths', engine)
    clean_states_df.to_sql('states', engine)
    merged_df.to_sql('merged', engine)
except:
    print("Table Creation Error maybe already exists")
    


Table Creation Error maybe already exists


In [19]:
# Add primary key to "merged" table that contains all cleaned data
try:   
    with engine.connect() as con:
        con.execute('ALTER TABLE merged ADD PRIMARY KEY ("State", "Date");')    
except:
    print("Primary key most likely addedd")

Primary key most likely addedd


In [20]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import MetaData

# Using SQL Alchemy to Query DB 
Base = automap_base()
Base.prepare(engine, reflect=True)
metadata = MetaData()
session = Session(engine)

Merged = Base.classes.merged



In [22]:
from datetime import datetime as dt
from sqlalchemy import desc, asc, func 

# Get Results by Date
q = session.query(Merged).filter(Merged.Date == dt(2021, 2, 1))
q.all()


# Filter Max column
qry = q.order_by(desc(Merged.Number_of_Cases))
for row in qry:
    print(row.State, row.Cumulative_Prisoner_Deaths)

California 40908.0
Texas 36539.0
Florida 27129.0
New York 35319.0
Illinois 21273.0
Ohio 11230.0
Georgia 14242.0
Pennsylvania 21687.0
North Carolina 9342.0
Arizona 13124.0
Tennessee 9753.0
New Jersey 21513.0
Indiana 9989.0
Michigan 15536.0
Wisconsin 6436.0
Massachusetts 14607.0
Virginia 6474.0
Missouri 6748.0
Minnesota 6202.0
Alabama 7688.0
South Carolina 7283.0
Louisiana 8912.0
Colorado 5641.0
Oklahoma 3564.0
Kentucky 3780.0
Maryland 7154.0
Utah 1668.0
Iowa 4906.0
Washington 4285.0
Arkansas 4895.0
Kansas 3809.0
Nevada 4278.0
Mississippi 6056.0
Connecticut 7119.0
Nebraska 1920.0
New Mexico 3295.0
Idaho 1725.0
Oregon 1957.0
Puerto Rico None
West Virginia 2028.0
Rhode Island 2173.0
South Dakota 1778.0
North Dakota 1447.0
Montana 1234.0
Delaware 1101.0
New Hampshire 1059.0
Alaska 262.0
Wyoming 596.0
Maine 595.0
District of Columbia 916.0
Hawaii 410.0
Vermont 175.0
Guam None
Virgin Islands None
Northern Mariana Islands None
